# Notebook for exploring the Chinese-English Buddhist Dictionary

In [1]:
import re

import pandas as pd
import numpy as np

from curation_util import ExtractWords
from curation_util import P2englishPN

# Load words table
colnames = ['id', 'simplified', 'traditional', 'pinyin', 
            'english', 'grammar', 'concept_cn', 'concept_en', 
            'domain_cn', 'domain_en', 'subdomain_cn', 'subdomain_en',
            'image', 'mp3', 'notes', 'headword']
types = {'id': np.uint32, 'simplified': unicode, 'traditional': unicode, 
        'pinyin': np.object, 'english': np.object, 'grammar': np.object,
        'concept_cn': np.object, 'concept_en': np.object,
        'domain_cn': np.object, 'domain_en': np.object, 'subdomain_cn': np.object,
         'subdomain_en': np.object, 'image': np.object, 'mp3': np.object,
         'notes': np.object, 'headword': np.uint32}
index_col = ['headword', 'id']
df = pd.read_table('../data/dictionary/words.txt', names=colnames, dtype=types,
                  index_col = index_col)
cols = ["simplified", "traditional", "pinyin", "english", "notes"]
df[df.simplified == '佛'][cols]

simplified traditional pinyin                 english  \
headword id                                                           
3618     3618          佛          \N     fó   Buddha / Awakened One   
         3619          佛          \N     fó  Buddhist / of Buddhism   

                                                           notes  
headword id                                                       
3618     3618  Sanskrit: buddha, Pali: buddha, Japanese: buts...  
         3619  As a modifier describing a concept that relate...

In [3]:
# Conversions for a word based on a traditional characters
# Input
luid = 46083
#tradArr = ["大", "集會", "正法", "經"]
tradArr = ExtractWords(u"賢劫經")
print "len(tradArr) = %d" % len(tradArr)
sanskrit = u"Sanghāṭīsūtradharmaparyāya"
english = u""
grammar = "proper noun"
concept = u"\\N\t\\N"
#concept = u"佛\tBuddha"
#concept = u"菩萨\tBodhisattva"
concept = u"经\tSūtra"
#subdomain = u"中国佛教\tChinese Buddhism"
#subdomain = u"\\N\t\\N"
subdomain = u"大乘佛教\tMahāyāna Buddhism"

# Generated
domain = u"佛教\tBuddhism"
#domain = u"古文\tClassical Chinese"
traditional = ""
simplified = ""
pinyin = ""
for t in tradArr:
  sDF = df[df.traditional == t]['simplified']
  tDF = df[df.traditional == t]['traditional']
  pDF = df[df.traditional == t]['pinyin']
  if sDF.count() > 0:
    simplified += sDF.iloc[0].decode('utf-8')
    traditional +=tDF.iloc[0].decode('utf-8')
    pinyin += pDF.iloc[0].decode('utf-8') + " "
  else:
    sDF = df[df.simplified == t]['simplified']
    tDF = df[df.simplified == t]['simplified']
    pDF = df[df.simplified == t]['pinyin']
    if sDF.count() > 0:
      simplified += sDF.iloc[0].decode('utf-8')
      traditional += sDF.iloc[0].decode('utf-8')
      pinyin += pDF.iloc[0].decode('utf-8') + " "
    else:
      print "%s not found" % t
pinyin.strip()
print simplified
trad = traditional
if simplified == traditional:
  trad = "\\N"

# Do some guessing on the best English name
if english == "":
  english = P2englishPN(pinyin)
if sanskrit != "":
  if english != "":
    english = u"%s (%s)" % (sanskrit, english)
  else:
    english = u"%s" % sanskrit

print trad
print pinyin
print english

len(tradArr) = 2
贤劫经
賢劫經
xián jié jīng 
Sanghāṭīsūtradharmaparyāya (Xian Jie Jing)


In [59]:
# Template for a dictionary entry for a Buddhist term
# Input
tid = 424
kid =1412

pali = u""
japanese = u""

# References
abc = u"ABC '%s'" % pinyin
bl = u"BL '%s'" % sanskrit
bscd = u"BCSD '%s'" % traditional
ccd = u"CCD '%s'" % simplified
fe = u"FE '%s'" % traditional
fgdb = u"FGDB '%s'" % traditional
gced = u"GCED '%s'" % traditional[0]
ghdc = u"GHDC '%s'" % simplified
jebd = u"JEBD '%s'" % japanese
kdc = u"KDC %d" % kid
mw = u"MW '%s'" % sanskrit
ncced = u"NCCED '%s'" % simplified
t = u"T %d" % tid
refArr = [kdc, t] # Add based on references checked

# Generated
if grammar != "proper noun":
  pinyin = pinyin.replace(" ", "")

# Sanskrit, Pali, and Japanese equivalents
fromLang = ""
if sanskrit != "":
  fromLang = u"From Sanskrit: %s" % sanskrit
if pali != "":
  fromLang += u", Pali: %s" % pali
if japanese != "":
  fromLang += u", Japanese: %s" % japanese
if fromLang != "":
  fromLang += u"; "

# References
ref = u""
for r in refArr:
  if r != "":
    ref += r + "; "
ref = re.sub("; $", "", ref)
if ref != "":
  ref = u"(%s)" % ref

note = u"the name of a bodhisattva who is central in the 《阿差末菩薩經》Akṣayamatinirdeśasūtra in the Chinese Buddhist canon "

notes = u"%s%s%s" % (fromLang, note, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)

46082	大集会正法经	大集會正法經	dà jíhuì zhèngfǎ jīng 	Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing)	proper noun	经	Sūtra	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	From Sanskrit: Sanghāṭīsūtradharmaparyāya; the name of a bodhisattva who is central in the 《阿差末菩薩經》Akṣayamatinirdeśasūtra in the Chinese Buddhist canon (KDC 1412; T 424)	46082


In [60]:
# Template for a dictionary entry for a monastic
concept = u"法师\tMonastic"
daterange = u"490-589; "
notes = u"%sChinese monastic who translated sutras in the Chinese Buddhist canon %s" % (daterange, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)

46082	大集会正法经	大集會正法經	dà jíhuì zhèngfǎ jīng 	Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing)	proper noun	法师	Monastic	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	490-589; Chinese monastic who translated sutras in the Chinese Buddhist canon (KDC 1412; T 424)	46082


In [61]:
# Add a title from the Taisho to the dictionary and corpus
# Template for a Taisho title as a word entry in the dictionary

# Input
translator = u"Dānapāla"
daterange = u"1001"
genre = u"mahāsaṃnipāta"
url = u"http://tripitaka.cbeta.org/T13n0424"
nscrolls = 5
volume = 13
dynasty = u"Song"
datestr = u"2016-09-13"
concept = u"经\tSutra"

# Generated
notes = u"%sthe name of a text in the Chinese Buddhist canon %s" % (
    fromLang, ref)
print u"%d\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\\N\t\\N\t%s\t%d" % (
    luid, simplified, trad, pinyin, english, grammar, concept, domain, subdomain, notes, luid)
print

# Full title of the Taisho entry
title = u"%s 《%s》" % (english, traditional)
print u"          <tr>"
print u"            <td>%d</td>" % tid
print u"            <td><a href='/taisho/t0%d.html'>%s</a></td>" % (tid, title)
print u"          </tr>"
print

# Collection entry
entry = u"taisho/t0%d.csv\ttaisho/t0%d.html\t%s\tTranslated by %s\ttaisho/t0%d_00.txt\tTaishō\tSūtra\t%s\t%s" % (
    tid, tid, title, translator, tid, daterange, genre)
print entry
print

# Write list of scrolls to metadata file
scrollsDF = pd.DataFrame({"i": range(1, nscrolls + 1)})
genSource = lambda i: "taisho/t0%d_0%d.txt" % (tid, i) if (i < 10) else "taisho/t0%d_%d.txt" % (tid, i)
scrollsDF['source'] = scrollsDF['i'].map(genSource)
genOut = lambda i: "taisho/t0%d_0%d.html" % (tid, i) if (i < 10) else "taisho/t0%d_%d.html" % (tid, i)
scrollsDF['output'] = scrollsDF['i'].map(genOut)
genTitle = lambda i: "Scroll %d" % i
scrollsDF['title'] = scrollsDF['i'].map(genTitle)
filename = "../data/corpus/taisho/t0%d.csv" % tid 
cols = ['source', 'output', 'title']
scrollsDF.to_csv(filename, sep = "\t", header = False, columns = cols, index = False)

# Translation of colophon
kReference = u""
if kid != 0:
  if sanskrit != "":
    kReference = u"Sanskrit title and date "
  else:
    kReference = u"Date "
  kReference += u"%s from Lancaster (Lancaster 2004, 'K %d')" % (daterange, kid)
dynastyRef = u""
if dynasty != u"":
  dynastyRef = u"Translated by %s in the %s in %d scroll(s)" % (translator, dynasty, nscrolls)
print u"Volume %d, No. %d" % (volume, tid)
print english
print dynastyRef
print
print "<h4>Notes</h4>"
if kid > 0:
  print
  print kReference
  print
print "English translations: None"
print
print "<h4>Primary Source</h4>"
print u"%s, 《%s》 '%s,' in <i>Taishō shinshū Daizōkyō</i> 《大正新脩大藏經》, in Takakusu Junjiro, ed., (Tokyo: Taishō Shinshū Daizōkyō Kankōkai, 1988), Vol. %d, No. %d, Accessed %s, <a href='%s'>%s</a>." % (
    translator, traditional, english, volume, tid, datestr, url, url)
print
print "<h4>References</h4>"
print """<ol><li>Lancaster, L.R. 2004, <i>The Korean Buddhist Canon: A Descriptive Catalogue</i>, <a href="http://www.acmuller.net/descriptive_catalogue/">http://www.acmuller.net/descriptive_catalogue</a>.</li></ol>"""

46082	大集会正法经	大集會正法經	dà jíhuì zhèngfǎ jīng 	Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing)	proper noun	经	Sutra	佛教	Buddhism	大乘佛教	Mahāyāna Buddhism	\N	\N	From Sanskrit: Sanghāṭīsūtradharmaparyāya; the name of a text in the Chinese Buddhist canon (KDC 1412; T 424)	46082

          <tr>
            <td>424</td>
            <td><a href='/taisho/t0424.html'>Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing) 《大集會正法經》</a></td>
          </tr>

taisho/t0424.csv	taisho/t0424.html	Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing) 《大集會正法經》	Translated by Dānapāla	taisho/t0424_00.txt	Taishō	Sūtra	1001	mahāsaṃnipāta

Volume 13, No. 424
Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing)
Translated by Dānapāla in the Song in 5 scroll(s)

<h4>Notes</h4>

Sanskrit title and date 1001 from Lancaster (Lancaster 2004, 'K 1412')

English translations: None

<h4>Primary Source</h4>
Dānapāla, 《大集會正法經》 'Sanghāṭīsūtradharmaparyāya (Da Jihui Zhengfa Jing),' in <i>Taishō shinshū Daizōkyō</i> 《大正新脩大藏經》, in 